<h1 style="text-align: center; color: green">Nhập thư viện</h1>

In [1]:
import math
import pandas as pd
import sqlalchemy
from mysql.connector import Error, connect
from sklearn.model_selection import train_test_split

<h1 style="text-align: center; color: green">Các hàm cần dùng</h1>

In [2]:
def mse(x, y):
    summation = 0
    n = len(y)
    for i in range(0, n):
        difference = y[i] - x[i]
        squared_difference = difference ** 2
        summation = summation + squared_difference
    return summation / n


def correlation(x, y):
    mean_x = sum(x) / float(len(x))
    mean_y = sum(y) / float(len(y))

    sub_x = [i - mean_x for i in x]

    sub_y = [i - mean_y for i in y]
    numerator = sum([sub_x[i] * sub_y[i] for i in range(len(sub_x))])

    std_deviation_x = sum([sub_x[i] ** 2.0 for i in range(len(sub_x))])
    std_deviation_y = sum([sub_y[i] ** 2.0 for i in range(len(sub_y))])

    denominator = (std_deviation_x * std_deviation_y) ** 0.5
    cor = numerator / denominator
    return cor


def loadData(path):
    data = pd.read_csv(path)
    train_size = int(len(data) * 0.9)
    train_df = data[:train_size:]
    test_df = data[train_size:]
    return train_df, test_df, data.keys()


def calcDistancs(pointA, pointB, numOfFeature):
    tmp = 0
    for i in range(numOfFeature):
        tmp += (float(pointA[i]) - float(pointB[i])) ** 2
    return math.sqrt(tmp)


def kNearestNeighbor(x_train, y_train, point, k):
    distances = []
    for x, y in zip(x_train.values, y_train):
        distances.append({"label": y, "value": calcDistancs(x, point, len(x_train.columns))})
    distances.sort(key=lambda x: x["value"])
    labels = [value["label"] for value in distances]
    return labels[:k]


def findMostOccur(arr):
    labels = set(arr)
    ans = ""
    maxOccur = 0
    for label in labels:
        num = arr.count(label)
        if num > maxOccur:
            maxOccur = num
            ans = label
    return ans

<h1 style="text-align: center; color: green">Chia dữ liệu huấn luyện, dự đoán</h1>

In [3]:
df = pd.read_csv('dataset.csv')
x = df.drop(columns=['target'])
y = df['target']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, stratify=y, random_state=43)

<h1 style="text-align: center; color: green">Tính độ tương quan giữa các thuộc tính với nhãn</h1>

In [4]:
corr = []
corr_matrix = []

for i in x:
    corr.append({'feature': i, 'corr': correlation(x[i], y)})
corr = pd.DataFrame(data=corr, index=range(len(corr)), columns=['feature', 'corr'])

<h1 style="text-align: center; color: green">Dự đoán từ mô hình huấn luyện</h1>

In [5]:
numOfRightAnswer = 0
i = 0
for item, predict in zip(x_test.values, y_test):
    i += 1
    knn = kNearestNeighbor(x_train, y_train, item, 5)
    answer = findMostOccur(knn)
    numOfRightAnswer += predict == answer
    print("Thực tế: {} -> Dự đoán: {} -> Chính xác: {}".format(predict, answer, numOfRightAnswer / i))
print("Độ chính xác", numOfRightAnswer / len(y_test))

Thực tế: 0 -> Dự đoán: 0 -> Chính xác: 1.0
Thực tế: 1 -> Dự đoán: 1 -> Chính xác: 1.0
Thực tế: 1 -> Dự đoán: 1 -> Chính xác: 1.0
Thực tế: 1 -> Dự đoán: 1 -> Chính xác: 1.0
Thực tế: 0 -> Dự đoán: 0 -> Chính xác: 1.0
Thực tế: 0 -> Dự đoán: 0 -> Chính xác: 1.0
Thực tế: 1 -> Dự đoán: 1 -> Chính xác: 1.0
Thực tế: 0 -> Dự đoán: 0 -> Chính xác: 1.0
Thực tế: 0 -> Dự đoán: 1 -> Chính xác: 0.8888888888888888
Thực tế: 1 -> Dự đoán: 1 -> Chính xác: 0.9
Thực tế: 1 -> Dự đoán: 1 -> Chính xác: 0.9090909090909091
Thực tế: 0 -> Dự đoán: 0 -> Chính xác: 0.9166666666666666
Thực tế: 1 -> Dự đoán: 1 -> Chính xác: 0.9230769230769231
Thực tế: 0 -> Dự đoán: 0 -> Chính xác: 0.9285714285714286
Thực tế: 1 -> Dự đoán: 1 -> Chính xác: 0.9333333333333333
Thực tế: 0 -> Dự đoán: 0 -> Chính xác: 0.9375
Thực tế: 1 -> Dự đoán: 1 -> Chính xác: 0.9411764705882353
Thực tế: 1 -> Dự đoán: 1 -> Chính xác: 0.9444444444444444
Thực tế: 1 -> Dự đoán: 1 -> Chính xác: 0.9473684210526315
Thực tế: 0 -> Dự đoán: 0 -> Chính xác: 0.95
T

<h1 style="text-align: center; color: green">Lưu trữ vào cơ sở dữ liệu MySql</h1>

In [6]:
#Tạo database
def create_db(host, user, password, database):
    try:
        con = connect(
            host=host,
            user=user,
            password=password,
        )
        cursor = con.cursor()
        cursor.execute("CREATE DATABASE " + database)
    except Error as e:
        print(e)
    finally:
        cursor.close()
        con.close()


#Tạo bảng
def create_tbl(host, user, password, database, table, dataFrame):
    database_connection = sqlalchemy.create_engine(
        'mysql+mysqlconnector://{0}:{1}@{2}/{3}'.format(user, password, host, database))
    dataFrame.to_sql(con=database_connection, name=table, if_exists='replace')


create_db('localhost', 'root', '1111', "btl_iot")
create_tbl('localhost', 'root', '1111', "btl_iot", "corr_tbl", corr)